In [59]:
import importlib
from functools import partial

from torch import nn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

from decision_learning.data.shortest_path_grid import genData
from decision_learning.modeling.loss import SPOPlus
from decision_learning.modeling.models import LinearRegression
from decision_learning.modeling.train import train, calc_test_regret

In [23]:
# Only for on the fly reloading of changing modules
# import decision_learning.data.shortest_path_grid
# importlib.reload(decision_learning.data.shortest_path_grid)
# import decision_learning.modeling.val_metrics
# importlib.reload(decision_learning.modeling.val_metrics)

# import decision_learning.modeling.models
# importlib.reload(decision_learning.modeling.models)
# from decision_learning.modeling.models import LinearRegression
# import decision_learning.modeling.loss
# importlib.reload(decision_learning.modeling.loss)
# from decision_learning.modeling.loss import SPOPlus
# import decision_learning.modeling.train
# importlib.reload(decision_learning.modeling.train)
# from decision_learning.modeling.train import train, calc_test_regret

<module 'decision_learning.modeling.train' from '/home1/yongpeng/decision-focused-learning/src/decision_learning/modeling/train.py'>

# Optimization Solver

In [3]:
def shortest_path_solver(costs, size, sens = 1e-4):
    # Forward Pass
    starting_ind = 0
    starting_ind_c = 0
    samples = costs.shape[0]
    V_arr = torch.zeros(samples, size ** 2)
    for i in range(0, 2 * (size - 1)):
        num_nodes = min(i + 1, 9 - i)
        num_nodes_next = min(i + 2, 9 - i - 1)
        num_arcs = 2 * (max(num_nodes, num_nodes_next) - 1)
        V_1 = V_arr[:, starting_ind:starting_ind + num_nodes]
        layer_costs = costs[:, starting_ind_c:starting_ind_c + num_arcs]
        l_costs = layer_costs[:, 0::2]
        r_costs = layer_costs[:, 1::2]
        next_V_val_l = torch.ones(samples, num_nodes_next) * float('inf')
        next_V_val_r = torch.ones(samples, num_nodes_next) * float('inf')
        if num_nodes_next > num_nodes:
            next_V_val_l[:, :num_nodes_next - 1] = V_1 + l_costs
            next_V_val_r[:, 1:num_nodes_next] = V_1 + r_costs
        else:
            next_V_val_l = V_1[:, :num_nodes_next] + l_costs
            next_V_val_r = V_1[:, 1:num_nodes_next + 1] + r_costs
        next_V_val = torch.minimum(next_V_val_l, next_V_val_r)
        V_arr[:, starting_ind + num_nodes:starting_ind + num_nodes + num_nodes_next] = next_V_val

        starting_ind += num_nodes
        starting_ind_c += num_arcs

    # Backward Pass
    starting_ind = size ** 2
    starting_ind_c = costs.shape[1]
    prev_act = torch.ones(samples, 1)
    sol = torch.zeros(costs.shape)
    for i in range(2 * (size - 1), 0, -1):
        num_nodes = min(i + 1, 9 - i)
        num_nodes_next = min(i, 9 - i + 1)
        V_1 = V_arr[:, starting_ind - num_nodes:starting_ind]
        V_2 = V_arr[:, starting_ind - num_nodes - num_nodes_next:starting_ind - num_nodes]

        num_arcs = 2 * (max(num_nodes, num_nodes_next) - 1)
        layer_costs = costs[:, starting_ind_c - num_arcs: starting_ind_c]

        if num_nodes < num_nodes_next:
            l_cs_res = ((V_2[:, :num_nodes_next - 1] - V_1 + layer_costs[:, ::2]) < sens) * prev_act
            r_cs_res = ((V_2[:, 1:num_nodes_next] - V_1 + layer_costs[:, 1::2]) < sens) * prev_act
            prev_act = torch.zeros(V_2.shape)
            prev_act[:, :num_nodes_next - 1] += l_cs_res
            prev_act[:, 1:num_nodes_next] += r_cs_res
        else:
            l_cs_res = ((V_2 - V_1[:, :num_nodes - 1] + layer_costs[:, ::2]) < sens) * prev_act[:, :num_nodes - 1]
            r_cs_res = ((V_2 - V_1[:, 1:num_nodes] + layer_costs[:, 1::2]) < sens) * prev_act[:, 1:num_nodes]
            prev_act = torch.zeros(V_2.shape)
            prev_act += l_cs_res
            prev_act += r_cs_res
        cs = torch.zeros(layer_costs.shape)
        cs[:, ::2] = l_cs_res
        cs[:, 1::2] = r_cs_res
        sol[:, starting_ind_c - num_arcs: starting_ind_c] = cs

        starting_ind = starting_ind - num_nodes
        starting_ind_c = starting_ind_c - num_arcs
    # Dimension (samples, num edges)
    obj = torch.sum(sol * costs, axis=1)
    # Dimension (samples, 1)
    return sol, obj.reshape(-1,1)

# Initialize Data

data parameters

In [4]:
indices_arr = torch.randperm(100000)
indices_arr_test = torch.randperm(100000)

sim = 0
n_arr = [200, 400, 800, 1600]
ep_arr = ['unif', 'normal']
trials = 100

exp_arr = []
for n in n_arr:
    for ep in ep_arr:
        for t in range(trials):
            exp_arr.append([n, ep, t])


In [5]:
exp = [1600, 'unif', 10] #exp_arr[sim]
print(exp)

[1600, 'unif', 10]


In [24]:
ep_type = exp[1]
trial = exp[2]

# generate data
grid = (5, 5)  # grid size
num_data = exp[0]  # number of training data
num_feat = 5  # size of feature
deg = 6  # polynomial degree
e = .3  # noise width

# path planting for shortest path example
planted_good_pwl_params = {'slope0':0, 
                    'int0':2,
                    'slope1':0, 
                    'int1':2}
planted_bad_pwl_params = {'slope0':4, 
                    'int0':0,
                    'slope1':0, 
                    'int1':2.2}

plant_edge = True

In [25]:
generated_data = genData(num_data=num_data+200,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
sol, obj = shortest_path_solver(costs=generated_data['cost'], size=5)
final_data = {'X':generated_data['feat'],
              'true_cost':generated_data['cost'],
              'true_sol':sol,
              'true_obj':obj}

# test data
generated_data_test = genData(num_data=10000,
        num_features=num_feat, 
        grid=grid, 
        deg=deg, 
        noise_type=ep_type,
        noise_width=e,
        seed=indices_arr[trial],     
        plant_edges=plant_edge,
        planted_good_pwl_params=planted_good_pwl_params,
        planted_bad_pwl_params=planted_bad_pwl_params)
sol_test, obj_test = shortest_path_solver(costs=generated_data_test['cost'], size=5)
final_data_test = {'X':generated_data_test['feat'],
              'true_cost':generated_data_test['cost'],
              'true_sol':sol_test,
              'true_obj':obj_test}

2024-11-06 09:09:08,835 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-06 09:09:08,835 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 10 11 12 13 14 17 18 19 20 21 22 25 26 27 28 29 32 33 34 37]
2024-11-06 09:09:08,836 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-06 09:09:08,836 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.0
2024-11-06 09:09:08,836 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-06 09:09:08,836 - decision_learning.data.shortest_path_grid - DEBUG - chg_pt: 0.55
2024-11-06 09:09:08,868 - decision_learning.data.shortest_path_grid - DEBUG - good_bad_edges: [ 1  4  9 16 24 31 36 39  0  3  8 15 23 30 35 38], remain_edges: [ 2  5  6  7 

Split data into train and validation set

In [26]:
# Splitting each input in the same way
train_dict = {}
val_dict = {}

for key, value in final_data.items():
    train_data, val_data = train_test_split(value, test_size=0.2, random_state=42)
    train_dict[key] = train_data
    val_dict[key] = val_data


# double checking the splitting is done correctly
# Splitting indices
indices = np.arange(len(next(iter(final_data.values()))))
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

for key in final_data.keys():
    train_same = (final_data[key][train_indices] == train_dict[key]).all()
    val_same = (final_data[key][val_indices] == val_dict[key]).all()
    print(f'{key}: {train_same}, {val_same}')

X: True, True
true_cost: True, True
true_sol: True, True
true_obj: True, True


# Training Loop

Initialize Inputs to Trainining Loop

In [31]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = SPOPlus(optmodel=optmodel)

# training, validation data
train_data_dict = train_dict
val_data_dict = val_dict

In [32]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=True)

100%|██████████| 12/12 [00:00<00:00, 3195.25it/s]
2024-11-06 09:10:09,493 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 5.798262405395508, val_metric: 0.1870353866853607
2024-11-06 09:10:09,493 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 5.798262405395508, val_metric: 0.1870353866853607
100%|██████████| 12/12 [00:00<00:00, 3424.85it/s]
2024-11-06 09:10:09,578 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 4.5747345553504095, val_metric: 0.19589737842479885
2024-11-06 09:10:09,578 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 4.5747345553504095, val_metric: 0.19589737842479885
100%|██████████| 12/12 [00:00<00:00, 3454.00it/s]
2024-11-06 09:10:09,662 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 4.560629161198934, val_metric: 0.1734712046402459
2024-11-06 09:10:09,662 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 4.560629161198934, val_metric: 0.1734712046402459
100%|█

In [33]:
test_regret = calc_test_regret(pred_model=pred_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.06759093793637577


## MSE

In [27]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = nn.MSELoss()

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'target':train_dict['true_cost']}) # extra input key needed for loss function

val_data_dict = val_dict

In [29]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=True)

100%|██████████| 12/12 [00:00<00:00, 3463.98it/s]
2024-11-06 09:09:21,646 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 1.8876804921362136, val_metric: 0.2392113968593179
2024-11-06 09:09:21,646 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 1.8876804921362136, val_metric: 0.2392113968593179
100%|██████████| 12/12 [00:00<00:00, 3477.38it/s]
2024-11-06 09:09:21,683 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.765940973493788, val_metric: 0.2152039387182229
2024-11-06 09:09:21,683 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.765940973493788, val_metric: 0.2152039387182229
100%|██████████| 12/12 [00:00<00:00, 3445.72it/s]
2024-11-06 09:09:21,720 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.6751075949933794, val_metric: 0.21910987352311345
2024-11-06 09:09:21,720 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.6751075949933794, val_metric: 0.21910987352311345
100%

In [30]:
test_regret = calc_test_regret(pred_model=pred_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.08562925913696434


## Cosine Loss

In [40]:
# Prediction Model
pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                 output_dim=train_dict['true_cost'].shape[1])

# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# loss function
loss_fn = nn.CosineEmbeddingLoss()

# training, validation data
train_data_dict = train_dict
train_data_dict.update({'input2':train_dict['true_cost'], 
                       'target':torch.ones(train_dict['true_cost'].shape[0])}) # extra input key needed for loss function

val_data_dict = val_dict

In [41]:
metrics, trained_model = train(pred_model=pred_model,
                optmodel=optmodel,
                loss_fn=loss_fn,
                train_data_dict=train_data_dict,
                val_data_dict=val_data_dict,
                num_epochs=100,
                lr=0.1,
                scheduler_params={'step_size': 10, 'gamma': 0.1},
                minimization=True)

100%|██████████| 12/12 [00:00<00:00, 3458.51it/s]
2024-11-06 09:18:08,838 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 0.07857334117094676, val_metric: 0.15394632031310854
2024-11-06 09:18:08,838 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 0.07857334117094676, val_metric: 0.15394632031310854
100%|██████████| 12/12 [00:00<00:00, 3487.02it/s]
2024-11-06 09:18:08,884 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.023548006556100315, val_metric: 0.16055482282145492
2024-11-06 09:18:08,884 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 0.023548006556100315, val_metric: 0.16055482282145492
100%|██████████| 12/12 [00:00<00:00, 3498.65it/s]
2024-11-06 09:18:08,931 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.022603477413455645, val_metric: 0.1482329970276144
2024-11-06 09:18:08,931 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 0.022603477413455645, val_metric: 0.1482329

In [42]:
test_regret = calc_test_regret(pred_model=pred_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)
print(test_regret)

0.06871183614832832


## PG Loss

In [56]:
import decision_learning.modeling.loss
importlib.reload(decision_learning.modeling.loss)
from decision_learning.modeling.loss import PG_Loss

In [50]:
# optimization solver
optmodel = partial(shortest_path_solver,size=5)

# training, validation data
train_data_dict = train_dict
val_data_dict = val_dict

PG Loss Parameters

In [54]:
h_arr = [num_data**-.125, num_data**-.25, num_data**-.5, num_data**-1]
fin_diff_sch = ['B', 'F', 'C']

In [57]:
# loop over PG loss params
PG_results = []
for h in h_arr:
    for f in fin_diff_sch:
        
        print(h, f)
        
        # --- Need to instantiate new loss function and prediction model for each possible setting of PG loss ---   
        # loss function
        loss_fn = PG_Loss(optmodel=optmodel,
                         h=h,
                         finite_diff_type=f)
        # Prediction Model
        pred_model = LinearRegression(input_dim=train_dict['X'].shape[1],
                         output_dim=train_dict['true_cost'].shape[1])
        # ----------------------------------------------------------------------------------------------------------------
        
        metrics, trained_model = train(pred_model=pred_model,
                        optmodel=optmodel,
                        loss_fn=loss_fn,
                        train_data_dict=train_data_dict,
                        val_data_dict=val_data_dict,
                        num_epochs=100,
                        lr=0.1,
                        scheduler_params={'step_size': 10, 'gamma': 0.1},
                        minimization=True)
        test_regret = calc_test_regret(pred_model=pred_model,
                               test_data_dict=final_data_test,
                               optmodel=optmodel)

        # add current setting results to overall results
        cur_res = {'h':h, 'finite_diff_sch': f, 'test_regret': test_regret}
        PG_results.append(cur_res)

# convert to pandas dataframe
PG_results = pd.DataFrame(PG_results)


0.3976353643835253 B


100%|██████████| 12/12 [00:00<00:00, 3474.98it/s]
2024-11-06 10:50:49,084 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 23.011095428466795, val_metric: 0.6255135366346355
2024-11-06 10:50:49,084 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 23.011095428466795, val_metric: 0.6255135366346355
100%|██████████| 12/12 [00:00<00:00, 3451.16it/s]
2024-11-06 10:50:49,215 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 23.265939500596787, val_metric: 0.6662045242176788
2024-11-06 10:50:49,215 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 23.265939500596787, val_metric: 0.6662045242176788
100%|██████████| 12/12 [00:00<00:00, 3441.72it/s]
2024-11-06 10:50:49,345 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 23.35850109524197, val_metric: 0.6505704453340708
2024-11-06 10:50:49,345 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 23.35850109524197, val_metric: 0.6505704453340708
100%|█

0.3976353643835253 F


100%|██████████| 12/12 [00:00<00:00, 3494.53it/s]
2024-11-06 10:51:02,077 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.870707575480143, val_metric: 0.663972930745743
2024-11-06 10:51:02,077 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.870707575480143, val_metric: 0.663972930745743
100%|██████████| 12/12 [00:00<00:00, 3478.83it/s]
2024-11-06 10:51:02,207 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 22.582617653740776, val_metric: 0.6422090023064015
2024-11-06 10:51:02,207 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 22.582617653740776, val_metric: 0.6422090023064015
100%|██████████| 12/12 [00:00<00:00, 3482.20it/s]
2024-11-06 10:51:02,336 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 22.654586283365884, val_metric: 0.6620179739716615
2024-11-06 10:51:02,336 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 22.654586283365884, val_metric: 0.6620179739716615
100%|█

0.3976353643835253 C


...

0.000625 F


100%|██████████| 12/12 [00:00<00:00, 3539.25it/s]
2024-11-06 10:52:59,903 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.449234347873265, val_metric: 0.3697886319169076
2024-11-06 10:52:59,903 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.449234347873265, val_metric: 0.3697886319169076
100%|██████████| 12/12 [00:00<00:00, 3539.00it/s]
2024-11-06 10:53:00,036 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 18.69632191128201, val_metric: 0.3699657493483205
2024-11-06 10:53:00,036 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 18.69632191128201, val_metric: 0.3699657493483205
100%|██████████| 12/12 [00:00<00:00, 3537.26it/s]
2024-11-06 10:53:00,168 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 18.69419576856825, val_metric: 0.3757900466261107
2024-11-06 10:53:00,168 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 18.69419576856825, val_metric: 0.3757900466261107
100%|███

0.000625 C


100%|██████████| 12/12 [00:00<00:00, 3523.39it/s]
2024-11-06 10:53:12,740 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.136268191867405, val_metric: 0.5513897522055999
2024-11-06 10:53:12,740 - decision_learning.modeling.train - INFO - epoch: 0, train_loss: 21.136268191867405, val_metric: 0.5513897522055999
100%|██████████| 12/12 [00:00<00:00, 3541.49it/s]
2024-11-06 10:53:12,869 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 20.29042994181315, val_metric: 0.5520808371039588
2024-11-06 10:53:12,869 - decision_learning.modeling.train - INFO - epoch: 1, train_loss: 20.29042994181315, val_metric: 0.5520808371039588
100%|██████████| 12/12 [00:00<00:00, 3535.77it/s]
2024-11-06 10:53:12,998 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 20.282528347439236, val_metric: 0.5514522970361518
2024-11-06 10:53:12,998 - decision_learning.modeling.train - INFO - epoch: 2, train_loss: 20.282528347439236, val_metric: 0.5514522970361518
100%|█

NameError: name 'pd' is not defined

In [61]:
PG_results

,h,finite_diff_sch,test_regret
0,0.397635,B,0.766241
1,0.397635,F,0.703595
2,0.397635,C,0.760432
3,0.158114,B,0.742369
4,0.158114,F,0.732630
5,0.158114,C,0.732590
6,0.025000,B,0.715896
7,0.025000,F,0.711148
8,0.025000,C,0.703984
9,0.000625,B,0.572492


## FYL Loss